In [1]:
import numpy as np
from scipy.integrate import dblquad
from scipy import integrate
from matplotlib import pyplot
import time

# Fem_1D_Quad

In [ ]:
def fem1d_quad_Neumann(N, interval, c_func, f_func, bc, der_order):
    lower, upper = interval
    h = (upper - lower) / N

    def ref_fi(x, i):
        # reference function
        if x < 0.0 or x > 1.0:
            return 0.0
        if i == 0:
            return 2.0 * x**2 - 3.0 * x + 1.0
        if i == 1:
            return 2.0 * x**2 - x
        if i == 2:
            return -4.0 * x**2 + 4.0 * x
    def ref_fi_1(x, i):
        if i == 0:
            return 4.0 * x - 3.0
        if i == 1:
            return 4.0 * x - 1.0
        if i == 2:
            return -8.0 * x + 4.0

    def fi(x, i, xn):
        # local function
        return ref_fi( (x - xn) / h, i)
    def fi_1(x, i, xn):
        # chain derivation
        return ref_fi_1( (x - xn) / h , i) / h

    def A_func(x, alpha, beta, xn):
        # integrand for computing system matrix entries
        return c_func(x) * fi_1(x, alpha, xn) * fi_1(x, beta, xn)

    def B_func(x, beta, xn):
        # integrand for computing right-hand side entries
        return f_func(x) * fi(x, beta, xn)
    
    # create an array of mesh nodes
    P = np.array([lower + n * h for n in range(0, N + 1)])
    # create an array of nodes' index in per mesh  
    T = np.array([range(0, N), range(1, N + 1)])

    # set up the finite element nodes
    Nb = 2 * N + 1
    Pb = np.array([lower + k * h / 2 for k in range(0, Nb)])
    Tb = np.array([range(0, Nb - 2, 2), range(2, Nb, 2), range(1, Nb - 1, 2)])

    # initialize the system matrix
    A = np.zeros(shape=(Nb, Nb))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = P[T[0, n]]
        x_u = P[T[1, n]]
        # loop over all hat functions for current element
        for alpha in range(0, 3):
            for beta in range(0, 3):
                # integrate A_func over current element to compute system matrix entries
                r, _ = integrate.quad(A_func, x_b, x_u, args = (alpha, beta, x_b))
                A[Tb[beta, n], Tb[alpha, n]] += r

    if der_order == 0:
        A[-1, :] = 0.0
        A[-1, -1] = 1.0
    else:
        A[0, :] = 0.0
        A[0, 0] = 1.0

    direction = [-1.0, 1.0]
    # initialize the right-hand side
    B = np.zeros(shape=(Nb, 1))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = P[T[0,n]]
        x_u = P[T[1,n]]
        # loop over all hat functions for current element
        for beta in range(0, 3):
            # integrate B_func over current element to compute right-hand side entries
            r, _ = integrate.quad(B_func, x_b, x_u, args=(beta, x_b))
            B[Tb[beta, n], 0] += r
    B[:,0] += direction[der_order] * bc[der_order]*c_func(interval[der_order])
    # apply boundary conditions to right-hand side
    if der_order == 0:
        B[-1, 0] = bc[1]
    else:
        B[0, 0] = bc[0]

    # solve the linear system to obtain the solution coefficients
    coefficients = np.linalg.solve(A, B)

    # return solution coefficients, grid points, system matrix, and right-hand side
    return coefficients, Pb, A, B


In [ ]:
def fem1d_quad_Dirichlet(N, interval, c_func, f_func, bc):
    # unpack interval into variables b and u
    b, u = interval
    # compute size of each subinterval
    h = (u - b) / N

    def ref_fi(x, i):
        # reference function
        if x < 0.0 or x > 1.0:
            return 0.0
        if i == 0:
            return 2.0 * x**2 - 3.0 * x + 1.0
        if i == 1:
            return 2.0 * x**2 - x
        if i == 2:
            return -4.0 * x**2 + 4.0 * x
    def ref_fi_1(x, i):
        if i == 0:
            return 4.0 * x - 3.0
        if i == 1:
            return 4.0 * x - 1.0
        if i == 2:
            return -8.0 * x + 4.0

    def fi(x, i, xn):
        # local function
        return ref_fi( (x - xn) / h, i)
    def fi_1(x, i, xn):
        # chain derivation
        return ref_fi_1( (x - xn) / h , i) / h

    def A_func(x, alpha, beta, xn):
        # integrand for computing system matrix entries
        return c_func(x) * fi_1(x, alpha, xn) * fi_1(x, beta, xn)

    def B_func(x, beta, xn):
        # integrand for computing right-hand side entries
        return f_func(x) * fi(x, beta, xn)

    # create an array of mesh nodes
    P = np.array([b + n * h for n in range(0, N + 1)])
    # create an array of nodes' index in per mesh  
    T = np.array([range(0, N), range(1, N + 1)])

    # set up the finite element nodes
    Nb = 2 * N + 1
    Pb = np.array([b + k * h / 2 for k in range(0, Nb)])
    Tb = np.array([range(0, Nb - 2, 2), range(2, Nb, 2), range(1, Nb - 1, 2)])

    # initialize the system matrix
    A = np.zeros(shape=(Nb, Nb))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = P[T[0, n]]
        x_u = P[T[1, n]]
        # loop over all hat functions for current element
        for alpha in range(0, 3):
            for beta in range(0, 3):
                # integrate A_func over current element to compute system matrix entries
                r, _ = integrate.quad(A_func, x_b, x_u, args = (alpha, beta, x_b))
                A[Tb[beta, n], Tb[alpha, n]] += r

    # apply Dirichlet boundary conditions to system matrix
    A[0, :] = 0.0
    A[0, 0] = 1.0
    A[-1, :] = 0.0
    A[-1, -1] = 1.0
    
    # initialize the right-hand side
    B = np.zeros(shape=(Nb, 1))
    # loop over all mesh elements
    for n in range(0, N):
        # get endpoints of the current mesh element
        x_b = Pb[Tb[0,n]]
        x_u = Pb[Tb[1,n]]
        # loop over all hat functions for current element
        for beta in range(0, 3):
            # integrate B_func over current element to compute right-hand side entries
            r, _ = integrate.quad(B_func, x_b, x_u, args=(beta, x_b))
            B[Tb[beta, n], 0] += r

    # apply boundary conditions to right-hand side
    B[0, 0] = bc[0]
    B[-1, 0] = bc[1]

    # solve the linear system to obtain the solution coefficients
    coefficients = np.linalg.solve(A, B)

    # return solution coefficients, grid points, system matrix, and right-hand side
    return coefficients, Pb, A, B



In [ ]:
def c_func(x):
    return np.exp(x)
def f_func(x):
    return -np.exp(x) * (np.cos(x) - 2 * np.sin(x) - x * np.cos(x) - x * np.sin(x))

In [ ]:

for i in range(1, 4):
    n = 2**i
    interval = [0.0, 1.0];
    bc = [0.0, np.cos(1.0)]

    startTime = time.time()
    u, x, A, B= fem1d_quad_Dirichlet(n, interval, c_func, f_func, bc)
    endTime = time.time()

    u = u.squeeze()
    u_real = x * np.cos(x)
    pyplot.plot(x, u, '.')
    pyplot.plot(x, u_real)
    print(f"N: {n}, Time: {endTime - startTime :.5f}s, Error: {np.max( np.abs(u - u_real) )}" )

---

In [2]:
class RefFunc2d:
    '''
        triangle reference basis function\n
        only support poly = 1 or 2
    '''
    def __init__(self, poly : int):
        self.poly = poly 

    def __call__(self, input, i):
        return self.der(input, i, 0, 0)
        
    def der(self,input, i, x_der = 0, y_der = 0):
        input = np.array(input)
        if self.poly == 1:
            if x_der == 0 and y_der == 0:
                return self._poly_1(input, i)
            if x_der == 1 and y_der == 0:
                return self._poly_1_dx(input, i)
            if x_der == 0 and y_der == 1:
                return self._poly_1_dy(input, i)
            return 0.0
        if self.poly == 2:
            if x_der == 0 and y_der == 0:
                return self._poly_2(input, i)
            if x_der == 1 and y_der == 0:
                return self._poly_2_dx(input, i)
            if x_der == 0 and y_der == 1:
                return self._poly_2_dy(input, i)
            if x_der == 1 and y_der == 1:
                return self._poly_2_dxdy(input, i)
            if x_der == 2 and y_der == 0:
                return self._poly_2_dxdx(input, i)
            if x_der == 0 and y_der == 2:
                return self._poly_2_dydy(input, i)
            return 0.0


    def _poly_1(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        x, y = input
        if i == 0:
            return  -x -y + 1.0
        if i == 1:
            return x
        if i == 2:
            return y    
    def _poly_1_dx(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        if i == 0:
            return -1.0
        if i == 1:
            return 1.0
        if i == 2:
            return 0.0
    def _poly_1_dy(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        if i == 0:
            return -1.0
        if i == 1:
            return 0.0
        if i == 2:
            return 1.0
    
    def _poly_2(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        x, y = input
        if i == 0:
            return  2*(x**2 + y**2) + 4*x*y - 3*(x + y) + 1.0
        if i == 1:
            return 2*x**2 - x
        if i == 2:
            return 2*y**2 - y
        if i == 3:
            return -4*x**2 - 4*x*y + 4*x
        if i == 4:
            return 4*x*y
        if i == 5:
            return -4*y**2 - 4*x*y + 4*y 
    def _poly_2_dx(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        x, y = input
        if i == 0:
            return 4*x + 4*y - 3.0
            #return  2*(x**2 + y**2) + 4*x*y - 3*(x + y) + 1.0
        if i == 1:
            return 4*x - 1.0
            #return 2*x**2 - x
        if i == 2:
            return 0.0
            #return 2*y**2 - y
        if i == 3:
            return -8*x -4*y + 4.0
            #return -4*x**2 - 4*x*y + 4*x
        if i == 4:
            return 4*y
            #return 4*x*y
        if i == 5:
            return -4*y 
            #return -4*y**2 - 4*x*y + 4*y 
    def _poly_2_dy(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        x, y = input
        if i == 0:
            return 4*y + 4*x - 3.0
            #return  2*(x**2 + y**2) + 4*x*y - 3*(x + y) + 1.0
        if i == 1:
            return 0.0
            #return 2*x**2 - x
        if i == 2:
            return 4*y - 1.0
            #return 2*y**2 - y
        if i == 3:
            return -4*x
            #return -4*x**2 - 4*x*y + 4*x
        if i == 4:
            return 4*x
            #return 4*x*y
        if i == 5:
            return -8*y - 4*x + 4.0 
            #return -4*y**2 - 4*x*y + 4*y 

    def _poly_2_dxdy(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        if i == 0:
            return 4.0
            #return  2*(x**2 + y**2) + 4*x*y - 3*(x + y) + 1.0
        if i == 1:
            return 0.0
            #return 2*x**2 - x
        if i == 2:
            return 0.0
            #return 2*y**2 - y
        if i == 3:
            return -4.0
            #return -4*x**2 - 4*x*y + 4*x
        if i == 4:
            return 4.0
            #return 4*x*y
        if i == 5:
            return -4.0 
            #return -4*y**2 - 4*x*y + 4*y 
    def _poly_2_dxdx(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        if i == 0:
            return 4.0
            #return  2*(x**2 + y**2) + 4*x*y - 3*(x + y) + 1.0
        if i == 1:
            return 4.0
            #return 2*x**2 - x
        if i == 2:
            return 0.0
            #return 2*y**2 - y
        if i == 3:
            return -8.0
            #return -4*x**2 - 4*x*y + 4*x
        if i == 4:
            return 0.0
            #return 4*x*y
        if i == 5:
            return 0.0
            #return -4*y**2 - 4*x*y + 4*y 
    def _poly_2_dydy(self, input, i):
        if np.any(input < 0.0) or np.any(input > 1.0):
            return 0.0
        if i == 0:
            return 4.0
            #return  2*(x**2 + y**2) + 4*x*y - 3*(x + y) + 1.0
        if i == 1:
            return 0.0
            #return 2*x**2 - x
        if i == 2:
            return 4.0
            #return 2*y**2 - y
        if i == 3:
            return 0.0
            #return -4*x**2 - 4*x*y + 4*x
        if i == 4:
            return 0.0
            #return 4*x*y
        if i == 5:
            return -8.0
            #return -4*y**2 - 4*x*y + 4*y 

In [3]:
class LocFunc2d:
    def __init__(self, ref_func: RefFunc2d):
        self.ref_func = ref_func
    def __call__(self, input, i, Trans):
        return self.der(input ,i, Trans, 0 ,0)

    def der(self, input, i, Trans, x_der=0, y_der=0):
        X, Y = Trans
        input_hat, J = self._affine(input, Trans)
        if x_der == 0 and y_der == 0:
            return self.ref_func(input_hat, i)
        if x_der == 1 and y_der == 0:
            return (
            (Y[2]-Y[1]) * self.ref_func.der(input_hat, i, x_der=1, y_der=0)
            +
            (Y[0]-Y[1]) * self.ref_func.der(input_hat, i, x_der=0, y_der=1)
            ) / J
        if x_der == 0 and y_der == 1:
            return (
            (X[0]-X[2]) * self.ref_func.der(input_hat, i, x_der=1, y_der=0)
                +
            (X[1]-X[0]) * self.ref_func.der(input_hat, i, x_der=0, y_der=1)
            ) / J
        if x_der == 1 and y_der == 1:
            return (
            (X[0]-X[2])*(Y[2]-Y[0])*self.ref_func.der(input_hat,i,2,0)
                +
            (X[0]-X[2])*(Y[0]-Y[1])*self.ref_func.der(input_hat,i,1,1)
                +
            (X[1]-X[0])*(Y[2]-Y[0])*self.ref_func.der(input_hat,i,1,1)
                +
            (X[1]-X[0])*(Y[0]-Y[1])*self.ref_func.der(input_hat,i,0,2)
            ) / (J**2)
        if x_der == 2 and y_der == 0:
            return (
            (Y[2]-Y[0])**2 * self.ref_func.der(input_hat,i,2,0)
                +
            2*(Y[2]-Y[0])*(Y[0]-Y[1]) * self.ref_func.der(input_hat,i,1,1)
                +
            (Y[0]-Y[1])**2 * self.ref_func.der(input_hat,i,0,2)
            ) / (J ** 2)
        if x_der == 0 and y_der == 2:
            return (
            (X[0]-X[2])**2 * self.ref_func.der(input_hat,i,2,0)
                +
            2*(X[0]-X[2])*(X[1]-X[0]) * self.ref_func.der(input_hat,i,1,1)
                +
            (X[1]-X[0])**2 * self.ref_func.der(input_hat,i,0,2)
            ) / (J ** 2)
        return 0.0
    def _affine(self, input, T):
        x, y = input
        X, Y = T
        J = (X[1] - X[0])*(Y[2] - Y[0]) - (X[2] - X[0])*(Y[1] - Y[0])
        x_hat = ((Y[2] - Y[0])*(x - X[0]) - (X[2] - X[0])*(y - Y[0])) / J
        y_hat = (-(Y[1] - Y[0])*(x - X[0]) + (X[1] - X[0])*(y - Y[0])) / J
        return (x_hat, y_hat), J

In [4]:
class Mesh2d:
    def __init__(self, interval, Num, poly, bc):
        self.interval = interval
        self.Num = Num
        self.poly = poly
        self.bc = bc
        self.P, self.T = self._gen_P_T()
        self.Pb, self.Tb = self._gen_Pb_Tb()
        self.Be = self._gen_Be()
        self.Bn = self._gen_Bn()

    def get_info_matrices(self):
        return self.P, self.T, self.Pb, self.Tb, self.Be ,self.Bn
    
    def _gen_P_T(self):
        lb, ub = self.interval[:,0], self.interval[:,1]
        h = (ub - lb) / self.Num

        Nx, Ny = np.squeeze(self.Num)
        P = np.zeros(shape=(2, (Nx+1)*(Ny+1)))
        for i in range(Nx + 1):
            for j in range(Ny + 1):
                P[0, i * (Ny + 1) + j] = (lb[0] + i * h[0])
                P[1, i * (Ny + 1) + j] = (lb[1] + j * h[1])

        T = np.zeros(shape = (3, 2 * Nx * Ny), dtype=int)
        ref_T = np.array([[0,Ny+1,0+1],[0+1,Ny+1,Ny+1+1]],dtype=int).T
        for n in range(Nx * 2*Ny):
            i = n // (2 * Ny)
            j = n % (2 * Ny)
            j1, j2 = j // 2, j % 2
            T[:,n] = ref_T[:,j2] +  (Ny + 1) * i + j1

        return P, T
    def _gen_Pb_Tb(self):
        if self.poly == 1:
            return self._gen_P_T()
        # self.poly = 2
        Nx, Ny = np.squeeze(self.Num)
        lb, ub = self.interval[:,0], self.interval[:,1]
        h = (ub - lb) / (2 * self.Num)

        Pb = np.zeros(shape=(2, (2*Nx+1)*(2*Ny+1)))
        for i in range(2*Nx + 1):
            for j in range(2*Ny + 1):
                Pb[0, i * (2*Ny + 1) + j] = (lb[0] + i * h[0])
                Pb[1, i * (2*Ny + 1) + j] = (lb[1] + j * h[1])

        Tb = np.zeros(shape=(6, 2*Nx*Ny), dtype=int)
        ref_Tb = np.array([[0, 2*(2*Ny+1), 0+2, 2*Ny+1, 2*Ny+1+1, 1],
        [2, 2*(2*Ny+1), 2*(2*Ny+1)+2, 2*Ny+1+1, 2*(2*Ny+1)+1, 2*Ny+1+2]],
        dtype=int).T
        for n in range(2 * Nx * Ny):
            i = n // (2 * Ny)
            j = n % (2 * Ny)
            j1, j2 = j // 2, j % 2
            Tb[:,n] = ref_Tb[:,j2] + 2*(2*Ny+1) * i + 2 * j1
        return Pb, Tb
    def _gen_Be(self):
        #boundary edge info-matrix
        Nx, Ny = np.squeeze(self.Num)
        Be = np.zeros(shape=(4, 2*(Nx+Ny)),dtype=int)
        
        st_points = [[0, 0], [Nx - 1, 1], [Nx - 1, 2 * Ny - 1], [0, 2 * Ny - 2]]
        directions = [[1, 0], [0, 2], [-1, 0], [0, -2]]
        
        n = 0
        for idx in range(4):
            st_point = st_points[idx]
            direction = directions[idx]
            for k in range(self.Num[int(idx % 2)]):
                i = st_point[0] + k * direction[0]
                j = st_point[1] + k * direction[1]
                element_id = i * 2 * Ny + j
                #print(element_id)
                Be[0, n] = self.bc[idx]
                Be[1, n] = element_id
                if idx == 0:
                    Be[2:, n] = self.T[0:2, element_id]
                elif idx == 1:
                    Be[2:, n] = self.T[1:3, element_id]
                elif idx == 2 or idx == 3:
                    Be[2, n] = self.T[2, element_id]
                    Be[3, n] = self.T[0, element_id]
                n += 1
        return Be
    def _gen_Bn(self):
        #boundary node info-matrix
        Nx, Ny = np.squeeze(self.Num)
        if self.poly == 1:
            Bn = np.zeros(shape=(2,2*(Nx + Ny)), dtype=int)
            R = [0, Nx*(Ny+1), Nx*(Ny+1) + Ny, Ny, 0]
            S = [Ny+1, 1, -(Ny+1), -1]
            D = [Nx, Ny, Nx, Ny]
            
        elif self.poly == 2:
            Bn = np.zeros(shape=(2, 4*(Nx + Ny)), dtype=int)
            R = [0, 2*Nx*(2*Ny+1), 2*Nx*(2*Ny+1) + 2*Ny, 2*Ny, 0]
            S = [2*Ny+1, 1, -(2*Ny+1), -1]
            D = [2*Nx, 2*Ny, 2*Nx, 2*Ny]
            
        start = end = 0
        for i in range(4):
            start = end
            end = end + D[i]
            Bn[0, start:end] = self.bc[i]
            Bn[1, start:end] = np.arange(R[i],R[i+1],S[i])        

        return Bn 

In [46]:
'''定义区间 以及 分割个数'''
interval = np.array([[0.0, 1.0], [0.0, 1.0]])# [x_min, x_max; y_min, y_max]
num = np.array([2, 2],dtype=int)# [x_num, y_num]
bc = [-1, -1, -1, -1]
'''剖分区域、参考函数 '''
poly = 2
ref_func = RefFunc2d(poly)
loc_func = LocFunc2d(ref_func)
mesh = Mesh2d(interval, num, poly, bc)

P, T, Pb, Tb, Be, Bn = mesh.get_info_matrices()


In [45]:
poly

1

In [47]:
def A_func(x, y, alpha, beta, Trans):
    return loc_func((x, y), alpha, Trans) * loc_func((x, y), beta, Trans)
    
N = T.shape[1]
Nb = Pb.shape[1]
A = np.zeros(shape=(Nb, Nb))
for n in range(0, N):
    X = P[0, T[:, n]]
    Y = P[1, T[:, n]]
    Trans = (X, Y)
    if n % 2 == 0:
        gfun = lambda x : Y[0]
        hfun = lambda x : (Y[2] - Y[1])/(X[2] - X[1]) * (x-X[2]) + Y[2] 
    else:
        gfun = lambda x : (Y[1] - Y[0])/(X[1] - X[0]) * (x-X[1]) + Y[1]
        hfun = lambda x : Y[0]
    for alpha in range(0, len(Tb)):
        for beta in range(0, len(Tb)):
            val, err = dblquad(func=A_func, a=X[0], b=X[1], gfun=gfun, hfun=hfun, args=(alpha, beta, Trans))
            A[Tb[beta, n], Tb[alpha, n]] += val
            print(val, err)

    
    

0.004166666666666666 7.281731646623692e-16
-0.0006944444444444443 1.8303354837454534e-16
-0.0006944444444444443 7.596255260171542e-17
5.963111948670274e-19 6.802229581552296e-16
-0.0027777777777777757 1.3544292760229468e-16
1.1926223897340549e-18 2.567711412564546e-16
-0.0006944444444444443 1.8303354837454534e-16
0.004166666666666667 7.281731646623687e-16
-0.0006944444444444445 7.596255260171544e-17
0.0 6.802229581552295e-16
1.6263032587282567e-19 2.5677114125645453e-16
-0.0027777777777777775 1.3544292760229485e-16
-0.0006944444444444443 7.596255260171542e-17
-0.0006944444444444445 7.596255260171544e-17
0.004166666666666667 2.940586392846614e-16
-0.002777777777777778 2.386278219311533e-16
1.2475668208918272e-19 1.9836577760603923e-16
-4.909219265948413e-19 1.9836577760603923e-16
5.963111948670274e-19 6.802229581552296e-16
0.0 6.802229581552295e-16
-0.002777777777777778 2.386278219311533e-16
0.02222222222222222 2.9285905726393385e-15
0.01111111111111111 6.031949339584762e-16
0.011111111

In [48]:
X, Y

(array([0. , 0.5, 0.5]), array([0.5, 0. , 0.5]))

In [33]:
i = 0
input = (0.5, 0.5)
print(ref_func.der(input,i,0,0))
print(ref_func.der(input,i,1,0))
print(ref_func.der(input,i,0,1))
print(ref_func.der(input,i,1,1))
print(ref_func.der(input,i,2,0))
print(ref_func.der(input,i,0,2))

0.0
1.0
1.0
4.0
4.0
4.0


In [35]:
A = [(0.0,1.0,0.0),(0.0,0.0,1.0)]
print(loc_func.der(input,i,A,0,0))
print(loc_func.der(input,i,A,1,0))
print(loc_func.der(input,i,A,0,1))
print(loc_func.der(input,i,A,1,1))
print(loc_func.der(input,i,A,2,0))
print(loc_func.der(input,i,A,0,2))

0.0
1.0
1.0
4.0
4.0
4.0
